# Pandas 2:  Cleaning data 

Probably the best thing about Pandas is its extensive toolset for managing data.  Here we describe features of Pandas that allow us to clean data that, for reasons beyond our control, comes in a form that's not immediately amendable to analysis. This is the first of several such notebooks.  

Outline:  

* [Want operator](#want).  Start with what we want to end up, then figure out how to get there.  
* [String methods](#strings).  Fixing string variables, especially strings that should really be numbers.  
* [Missing values](#missing).  Marking, dropping, counting missing values.    
* [Selecting variables and observations](#selection).  Choose the variables and observations we want by their labels.
* [Boolean selection](#boolean).  This is mostly what we do:  choose observations from conditions.  We use comparisons to produce Boolean variables and then use the Boolean variables to select observations that are `True`.  The next two methods extend this capability.  
* [The `isin` method](#isin).  Choose observations whose values are in lists you specify. 
* [The `contains` method](#contains).  Flag observations that contain a specific piece of text.  Another string method, operates through Booleans.  

<!--
* [The `query` method](#query).  Similar capability using database syntax.  This is one of many examples in which **SQL database** tools have been built into Pandas.  
--> 

<!--
* [Indexing](#index).  Setting and resetting the index.  Multi-indexes. 
* [Switching rows and columns](#pivot).  Transpose.  Pivot and melt.  
* [Stack and unstack](#stack).  Managing column structure and labels.  Melt.  
--> 

**Note: requires internet access to run.**  

<!-- 
internal links http://sebastianraschka.com/Articles/2014_ipython_internal_links.html
-->

This IPython notebook was created by Dave Backus, Chase Coleman, and Spencer Lyon for the NYU Stern course [Data Bootcamp](http://databootcamp.nyuecon.com/).  

<a id=prelims></a>

## Preliminaries 

In [ ]:
import sys                             # system module 
import pandas as pd                    # data package
import matplotlib.pyplot as plt        # graphics module  
import datetime as dt                  # date and time module
import numpy as np                     # foundation for Pandas 

%matplotlib inline                     

# check versions (overkill, but why not?)
print('Python version: ', sys.version)
print('Pandas version: ', pd.__version__)
print('Today: ', dt.date.today())

<a id=want></a>

## The want operator 

We need to know what we're trying to do -- what we **want** the data to look like.  We say we **apply the want operator**.  

Some problems we've run across that ask to be solved:

* Numerical data is contaminated by commas (marking thousands) or dollar signs.  
* Row and column labels are contaminated.  
* Missing values are marked erratically.  
* We have too much data, would prefer to choose a subset.  
* Variables run across rows rather than down columns.  

What we want in each case is the opposite of what we have:  we want nicely formatted numbers, clean row and column labels, and so on.  

We'll solve the first four problems here, the last one in the next notebook.  

### Example:  Chipotle data 

This data comes from a [New York Times story]((http://www.nytimes.com/interactive/2015/02/17/upshot/what-do-people-actually-order-at-chipotle.html) about the number of calories in a typical order at Chipotle.  The topic doesn't particularly excite us, but the data raises a number of issues that come up repeatedly. We adapt some code written by [Daniel Forsyth](http://www.danielforsyth.me/pandas-burritos-analyzing-chipotle-order-data-2/). 

In [ ]:
url = 'https://raw.githubusercontent.com/TheUpshot/chipotle/master/orders.tsv'
chp = pd.read_csv(url, sep='\t')   # tab (\t) delimited 
print('Variable dtypes:\n', chp.dtypes, sep='')
chp.head()

**Comment.** Note that the variable `item_price` has dtype object. The reason is evidently the dollar sign.  We'd prefer to have it as a number, specifically a float.  

### Example:  Data Bootcamp entry poll

This is the [poll](https://docs.google.com/forms/d/1N7ugrqIQNHm_e1BLVAtaRMnC8SmSiTDMoYBy_0FYZic/viewform) we did at the start of the course.  Responses were collected in a Google spreadsheet, which we converted to a csv and uploaded to our website.  

In [ ]:
pd.set_option("display.width", 80)

In [ ]:
import pandas as pd
url1 = 'http://pages.stern.nyu.edu/~dbackus/Data/'
url2 = 'Data-Bootcamp-entry-poll_s16.csv'
url = url1 + url2 
ep = pd.read_csv(url, header=0) 
print('Dimensions:', ep.shape)
print('\nData types:\n', ep.dtypes, sep='')
ep.head(2) 

**Comments.** This is mostly text data, which means it's assigned the dtype object.  Which is fine.  But there are two things that would make the data easier to work with:

* The column names are excessively verbose.  This one's easy:  We replace them with single words.  Which we do below.  
* The second one is harder.  Two of the questions -- social media and special topics -- say "mark all that apply."  In the spreadsheet, we have a list of every choice the person checked.  Our want is to count the number of each type of response.  For example, we might want a bar chart that gives us the number of each response.  The question is how we get there.  

In [ ]:
# rename variables 
newnames = ['time', 'program', 'career', 'programming', 'stats', 'media',
             'other', 'major', 'data', 'why', 'topics']
newnames = [name.title() for name in newnames]             
ep.columns = newnames  
ep.head()

In [ ]:
# check multi-response question to see what we're dealing with 
ep['Media'].head(20)

**Comment.**  Note the commas separating answers with more than one choice.  We want to unpack them somehow.  

### Example:  OECD healthcare statistics 

The OECD collects [healthcare data](http://www.oecd.org/els/health-systems/health-data.htm) on lots of (mostly rich) countries, which is helpful in producing comparisons.  Here we use a [spreadsheet](http://www.oecd.org/health/health-systems/OECD-Health-Statistics-2015-Frequently-Requested-Data.xls) linked in one of their documents. 

In [ ]:
url1 = 'http://www.oecd.org/health/health-systems/'
url2 = 'OECD-Health-Statistics-2015-Frequently-Requested-Data.xls'
docs = pd.read_excel(url1+url2, 
                     skiprows=3, 
                     usecols=[0, 51, 52, 53, 54, 55, 57],
                     sheetname='Physicians', 
#                     na_values=['..'],
                     skip_footer=21) 
print('Dimensions:', docs.shape)
print('\nIndex', docs.index.tolist(), sep='')
print('\nVariable dtypes:\n', docs.dtypes.tail(8), sep='')
docs.head()

**Comments.**  Here we have a couple issues.  

* The first column includes a space and a number:  `Australia 1`, `Chile 3`, etc.  We care about this because when we plot the data across countries, the country labels are going to be country names, so we want them in a better form than this.  
* The `..`'s in the sheet lead us to label any column that includes them as dtype object.  Here we want to label them as missing values.  
* If we want to plot each country against time, then we'll need to switch the rows and columns somehow, so that the x axis in the plot (the year) is the index and not the column label.  

One more thing before we proceeed:  change the name of the country variable. 

In [ ]:
names = list(docs)
docs = docs.rename(columns={names[0]: 'Country'})
docs.head(2)

### Example:  World Economic Outlook 

The IMF's [World Economic Outlook database](https://www.imf.org/external/ns/cs.aspx?id=28) contains a broad range of macroeconomic data for a large number of countries.  It's updated twice a year and is a go-to source for things like current account balances (roughly, the trade balance) and government debt and deficits.  It also has a few quirks, as we'll see.  

**Example.** Run the following code as is, and with the `thousands` and `na_values` parameters commented out.  How do the dtypes differ?

In [ ]:
url1 = 'http://www.imf.org/external/pubs/ft/weo/2015/02/weodata/'
url2 = 'WEOOct2015all.xls'
url = url1 + url2 
weo = pd.read_csv(url, sep='\t', 
                  usecols=[1,2,3,4,6,40,41,42,43,44], 
                  thousands=',', 
                  na_values=['n/a', '--']
                 )

print('Variable dtypes:\n', weo.dtypes, sep='')
weo.head()

**Comment.** This has several issues:  

* The variables run across rows with observations labeled 1980, 1981, etc across the top.  We saw the same problem in the previous example.    
* If we run the first version of the `read_csv` statement, the data columns (1980, 1981, etc) have dtype object.  A little work suggests that this is because they include commas marking thousands.   
* The entries labeled `n/a` need to be marked as missing values.

We can solve the last two in the `read_csv` function by deleting the hash -- which is what we see in the second `read_csv` statement.  The other one takes some work.  

**Question.**  Can we transpose the whole thing to get the data running down columns?  

In [ ]:
weo.T.head(10)

<a id='strings'></a>

## String methods 

We can treat variables as strings in Pandas in much the same way we dealt with strings in core Python.  Run the code below to remind yourself how this works.  

In [ ]:
dollars = '$123.45'
print('Type of variable dollars:', type(dollars))
num = dollars.replace('$', '')
num = float(num)
print('Type of variable num:', type(num))

**Pandas string methods.** We can do the same thing to all the observations of a variable with so-called **string methods**.  We append `.str` to a variable in a dataframe and then apply the string method of our choice.  If this is part of converting a number-like entry that has mistakenly been given dtype object, we then convert its dtype with the `astype` method.  

**Example.**  Let's use a string method to fix the `item_price` variable in the Chipotle dataframe.  This has three parts: 

* Use the method `str` to identify this as a string method.
* Apply the string method of our choice (here `replace`) to fix the string.
* Use the `astype` method to convert the fixed-up string to a float. 

We start by making a copy of the `chp` dataframe that we can experiment with.  

In [ ]:
chp.head()

In [ ]:
chpnum = chp.copy()
print('Original dtype:', chpnum['item_price'].dtype)
# create a copy of the df to play with 

In [ ]:
# delete dollar signs
chpnum['item_price'].str.replace('$', '').head()

In [ ]:
# delete dollar signs, convert to float, and assign back to chpnum 
chpnum['item_price'] = chpnum['item_price'].str.replace('$', '').astype(float)
print('New dtype:', chpnum['item_price'].dtype)

In [ ]:
# assign back to chp for future use 
chp = chpnum

**Comment.** We did everything here in one line:  replace the dollar sign with a string method, then converted to float using `astype`.  If you think this is too dense, you might break it into two steps.  

**Example.** Here we strip off the numbers at the end of the indexes in the OECD `docs` dataframe.  This involves some experimentation:  

* Play with the `rsplit` method to see how it works.  
* Apply `rsplit` to the example `country = 'United States 1'`.  
* Use a string method to do this to all the entries of the variable `Country`.  

In [ ]:
# try this with an example first 
country = 'United States 1'

# get documentation for the rsplit method
#country.rsplit?

In [ ]:
# an example 
country.rsplit()

**Comment.** Not quite, we only want to split once.  

In [ ]:
# what about this?
country.rsplit(maxsplit=1)

In [ ]:
# one more step, we want the first component of the list
country.rsplit(maxsplit=1)[0]

In [ ]:
docs["Country"].head()

In [ ]:
# now do this for the variable Country 
#docs['Country'].str.rsplit(maxsplit=1).str[0].head()   # explain why this doesn't work 
docs['Country'].str.rsplit(n=1).str[0].head() 

In [ ]:
# some people prefer the get method to slicing 
docs['Country'].str.rsplit(n=1).str.get(0).head()

In [ ]:
# now assign it to newdocs and see what we have 
newdocs = docs.copy()
newdocs['Country'] = newdocs['Country'].str.rsplit(n=1).str.get(0)
newdocs.head()

In [ ]:
# assign it back to docs for future use 
docs = newdocs 

**Comments.** 

* Note that we need two `str`'s here:  one to do the split, the other to extract the first element.  
* For reasons that mystify us, we ran into problems when we used `maxsplit=1`, but it works with `n=1`. 
* This is probably more than you want to know, but file away the possibilities in case you need them.  

<a id='missing'></a>

## Missing values 

It's important to label missing values, so that Pandas doesn't interpret entries as strings.  Pandas is also smart enough to ignore things labeled missing when it does calculations or graphs.  If we compute, for example, the mean of a variable, the default is to ignore missing values.  

We've seen that we can label certain entries as missing values in read statements:  `read_csv`, `read_excel`, and so on.  Here we do it directly, mostly to remind ourselves what's involved.  

### Marking missing values

**Example.** The `docs` dataframe contains a number of instances of `..` (double period).  How can we mark them as missing values?  

In [ ]:
docs = newdocs
docs.head()

**What to do.**  We use the replace method on the whole dataframe.  To mark something as missing, we replace it as `None`, which Pandas interprets as missing and labels `NaN`.  

In [ ]:
docs.replace(to_replace=['..'], value=[None]).head()

**Comment.**  Replace automatically updates the dtypes.  Here the double dots led us to label the variables as objects.  After the replace, they're now floats, as they should be.  

In [ ]:
docs.dtypes.head()

In [ ]:
docsna = docs.replace(to_replace=['..'], value=[None])
docsna.dtypes.head()

**Comment.**  Some people prefer to use the numpy `nan`.  Here's an example.  The only advantage is that we avoid possible conflicts with other uses of the value `None`.  

In [ ]:
docs.replace(to_replace=['..'], value=[np.nan]).head()

In [ ]:
# assign back to docs 
docs = docs.replace(to_replace=['..'], value=[np.nan])

**Comment.** Unlike the string methods we described earlier, this use of replace affects complete entries, not elements of string entries.  For example, suppose we tried to replace the periods in decimal numbers with an asterisk.  We could try the following, but it doesn't work:  the decimal numbers  don't change.  

In [ ]:
docs.replace(to_replace=['.'], value=['*']).head()

### Working with missing values 

In [ ]:
# grab a variable to play with
var = docsna[2013].head(10)
var

In [ ]:
# which ones are missing ("null")?
var.isnull()

In [ ]:
# which ones are not missing ("not null")?
var.notnull()

In [ ]:
# drop the missing 
var.dropna()

**Comment.** We usually don't have to worry about this, Pandas takes care of missing values automatically.  

**Comment.** Let's try a picture to give us a feeling of accomplishment. What else would you say we need?  How would we get it?  

In [ ]:
docs[2013].plot.barh(figsize=(4, 12)) 

<a id='selection'></a>

## Selecting variables and observations 

The word **selection** refers to choosing a subset of variables or observations using their labels or index.  Similar methods are sometimes referred to as **slicing**, **subsetting**, **indexing**, or **filtering**.  We'll treat the terms as synonymous.  

There are lots of ways to do this.  Mostly we do "Boolean" selection, which we address in the next section.  We review more direct options here, mostly at high speed because they're not things we use much.  

In the outline below, `df` is a dataframe, `var` and `varn` are variable names, `vlist = ['var1', 'var2']` is a list of variable names, and `nlist = [0, 3, 4]` is a list of numerical variable or observation indexes, and `n1` and `n2` are integers.  Some of the basic selection/indexing/slicing methods have the form:  

* `df[var]` extracts a variable -- a series, in other words.
* `df[var][3]` extracts observation 3 (starting at zero) from the series `df[var]`. 
* `df[vlist]` extracts a new dataframe consisting of the variables in `vlist`.  
* `df[nlist]` does the same thing.  
* `df[n1:n2]` extracts observations `n1` to `n2-1`, the traditional slicing syntax.  

We find the last one confusing:  it extracts rows, not columns.  Pandas guru Wes McKinney notes: "This might seem inconsistent to some readers."  Yup!  We don't do it much, partly for that reason.  

<!-- page 127 top -->

The Pandas docs push the `loc` and `iloc` methods.  We'll ignore them -- we don't use them much -- but if you're interested, see [the docs](http://pandas.pydata.org/pandas-docs/stable/indexing.html#different-choices-for-indexing).    

In [ ]:
# we create a small dataframe to experiment with 
small = weo.head()
small

**Exercise.** Try each of these in a different cell and explain what they do:  
    
* `small[['ISO', 'Units']]`
* `small[[0, 4]]`
* `small['2011']`
* `small['2011'][3]`
* `small[1:3]`

<a id='boolean'></a>

<a id='boolean'></a>

## Boolean selection  

This is mostly what we do:  we choose observations that satisfy one or more conditions.  We work through this one step at a time:  

* Example:  apply the want operator  
* Comparisons for dataframes 
* Boolean selection:  select observations for which the comparison is `True`
* The `isin` method

This is easier to describe with an example.  

### Example:  Apply the want operator to WEO

Our **want** here is to take the `weo` dataframe and extract government debt and deficits for a given set of countries.  Putting this to work involves several steps.  

Here's the head of the dataframe to give us a sense of what we're dealing with.  

In [ ]:
weo.head()

**Find variable and country codes.** Which ones do we want?  Let's start by seeing that's available.  Here we create special dataframes that include all the variables and their definitions and all the countries.  

Note the use of the `drop_duplicates` method, which does what it sounds like.    

In [ ]:
variable_list = weo[['WEO Subject Code', 'Subject Descriptor', 'Units']].drop_duplicates()
print('Number of variables: ', variable_list.shape[0])
variable_list.head()

In [ ]:
country_list = weo[['ISO', 'Country']].drop_duplicates()
print('Number of countries: ', country_list.shape[0])
country_list.head()

**Exercise.** 

* Construct a list of countries with `countries = weo[['ISO', 'Country']]`; that is, without applying the `drop_duplicates` method.  How large is it?  How many duplicates have we dropped?  
* What are the country codes (`ISO`) for Argentina, Germany, and Greece?  
* What are the variable codes (`WEO Subject Code`) for government debt (gross debt, percent of GDP) and net lending/borrowing (also percent of GDP)?  

**Comment.** Now that we have the country and variable codes, we can be more explicit about what we want.  We want observations with those country and variable codes.  

We work up to the solution one step at a time.  

### Comparisons for series 

We can construct comparisons for dataframe columns much as we did with simple variables.  The difference is that we get a complete column or True/False responses, not just one.  

Mutiple comparisons have a different syntax than we saw earlier.  `and` is replaced by `&`, and `or` is replaced by `|`.  And when we have more than comparison, we need to enclose them in parentheses.  

Here's an example.  

**Exercise.**  Compute and explain the comparisons:  

* `small['Units'] == 'National currency'`
* `small['2011'] >= 100`
* `(small['Units'] == 'National currency') & (small['2011'] >= 100)`
* `(small['Units'] == 'National currency') | (small['2011'] >= 100)`

### Boolean selection

Boolean selection simply chooses those observations for which a condition is `True`.  Some people refer to this as filtering.  

**Example.**  We choose obervations for which the units are `'National currency'`.  We do this first in two steps, then in one.  

In [ ]:
ncunits = small['Units'] == 'National currency'
small[ncunits]

In [ ]:
small[small['Units'] == 'National currency']

**Exercise.** Construct dataframes for which 

* `small['Units']` does **not** equal `'National currency'`.
* `small['Units']` equals `'National currency'` and `small['2011']` is greater than 100.  

<a id='isin'></a>

## The `isin` method

Pay attention now, this is really useful.  Suppose we want to extract the data for which `weo['ISO'] == 'ARG'` (Argentina) or `weo['ISO'] == 'GRC'` (Greece).  We could do that by combining the comparisons:  

```python
(weo['ISO'] == 'ARG') | (weo['ISO'] == 'GRC')
```

Remind youself that `|` stands for "or."  (What do we use for "and"?) 

A simpler approach is to apply the `isin` method to a variable.  This sets the comparison equal to `True` if the value of the observation is of `weo['ISO']` equals any element in a list.  We could do the same thing using mulitple comparisons, but this is a lot easier.  

Let's see how this works.  

**Example.**  Let's apply the same logic to variable codes.  If we want to extract the observations with codes 
```
vlist = ['GGXWDG_NGDP', 'GGXCNL_NGDP']
```

we would use 

In [ ]:
vlist = ['GGXWDG_NGDP', 'GGXCNL_NGDP']
weo['WEO Subject Code'].isin(vlist).head(45)

**Comment.** We're choosing 2 variables from 45, so there are lots of Falses.

In [ ]:
# this time let's use the result of isin for selection 
vlist = ['GGXWDG_NGDP', 'GGXCNL_NGDP']
weo[weo['WEO Subject Code'].isin(vlist)].head(6)

**Comment.** We can do the same thing with countries.  If we want to choose two variables and three countries, the code looks like:  

In [ ]:
variables = ['GGXWDG_NGDP', 'GGXCNL_NGDP']
countries = ['ARG', 'DEU', 'GRC']
weo_sub = weo[weo['WEO Subject Code'].isin(variables) & weo['ISO'].isin(countries)]
weo_sub

**Comments.**

* We've now done what we described when we applied the want operator. 
* This is a go-to method.  Circle it for later reference.  
* **This is a go-to method.  Circle it for later reference.**  

**Exercise.** Use the `isin` method to extract *Gross domestic product in US dollars* for China, India, and the United States.  Assign the result to the dataframe `gdp`.   

**Exercise (challenging).** Plot the variable `gdp['2015']` as a bar chart.  What would you say it needs?  

<a id='contains'></a>

## The `contains` method

Another useful one.  The `contains` string method for series identifies observations that contain a specific string.  If yes, the observation is labelled True, if no, False.  A little trick converts the True/False outcomes to ones and zeros.  

We apply it to the `Media` variable of the Entry Poll dataframe `ep`.  You may recall that this variable could have more than one response.  We tease them apart with the `contains` method.  Our want is to have a yes/no variable for each response.  

In [ ]:
# recall
ep['Media'].head(10)

In [ ]:
# the contains method
ep['Media'].str.contains('Twitter').head(10)

**Comment.** That's pretty good, we now know which students mentioned Twitter and which did not.  It's more useful, though, to convert this to zeros (False) and ones (True), which we do with this trick:  we multiply by 1.  

In [ ]:
ep['Media'].str.contains('Twitter').head(10)*1

**Comment.** Now let's do the same for some of the other entries and save them in new variables.  

In [ ]:
media = ['None', 'Twitter', 'Facebook', 'Blog'] 
oldep = ep.copy()

vnames = []
for x in media:
    newname = 'Media' + ':' + x 
    vnames.append(newname)
    ep[newname] = ep['Media'].str.contains(x)*1
    
vnames    

In [ ]:
media = ep[vnames]
media.head()

In [ ]:
media_counts = media.sum()
media_counts

In [ ]:
media_counts.plot.barh()

**Exercise.** What would you change in this graph?  How would you do it?  (Words are enough.) 

## Review

**Exercise.** We explore the Census's [Business Dynamics Statistics](http://www.census.gov/ces/dataproducts/bds/), a huge collection of data about firms. We've extracted a small piece of one of their databases that includes these variables for 2013:

* Size: size category of firms based on number of employees
* Firms: number of firms in each size category
* Emp: number of employees in each size category

Run the code cell below to load the data.  

In [ ]:
data = {'Size': ['a) 1 to 4', 'b) 5 to 9', 'c) 10 to 19', 'd) 20 to 49', 'e) 50 to 99',
                 'f) 100 to 249', 'g) 250 to 499', 'h) 500 to 999', 'i) 1000 to 2499',
                 'j) 2500 to 4999', 'k) 5000 to 9999', 'l) 10000+'], 
        'Firms': [2846416, 1020772, 598153, 373345, 115544, 63845,
                  19389, 9588, 6088, 2287, 1250, 1357], 
        'Emp': [5998912, 6714924, 8151891, 11425545, 8055535, 9788341, 
                6611734, 6340775, 8321486, 6738218, 6559020, 32556671]}
bds = pd.DataFrame(data) 
bds .head(3) 

Use the dataframe `bds` to:  

* Compute the mean number of firms with `nbar = bds['Firms'].mean()`. 
* Generate the new variable `bds['AboveMean']` that is `True` if the value of `bds['Firms']` is above the mean, `False` otherwise. 
* What dtype is this new variable?  
* Select the observations for which the number of firms is above the mean.  
* *Challenging.*  Fix the size categories.  Specifically, use a string method to eliminate the prefixes `a)`, `b)`, etc.  That is, change `a) 1 to 4` to `1 to 4`, `b) 5 to 9` to `5 to 9`, and so on.  *Hint:* Use the `split` method.  